In [1]:
#使用均方误差评估本章的算法
##########################监督学习###############################################
#广义线性模型：线性回归、岭回归、套索回归、对数几率回归
#K近邻（简单分类方法，将近邻的多个标签中出现次数最多的分配给该点）
#朴素贝叶斯，多项式朴素贝叶斯、高斯朴素贝叶斯
#决策树（从特征中学习，生成简单规则分割训练集，预测未知标签）
#支持向量机

In [2]:
###########################评估各回归算法##########################################
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import svm
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

df = pd.read_csv('./data/housing.csv',sep = ',',header=None)    #导入数据
df = df.iloc[np.random.permutation(len(df))]                     #打乱原数据顺序
X = df[df.columns[:-1]].values
Y = df[df.columns[-1]].values
#分别对线性回归、岭回归、套索回归和支持向量机（SVM）计算均方误差(MSE)和判定系数(R^2)
cv = 10                                                         #线性回归
print('linear regression')
lin = LinearRegression()
scores = model_selection.cross_val_score(lin,X,Y,cv = cv)
print('mean R2:%0.2f (+/- %0.2f)'%(scores.mean(),scores.std()*2))
predicted = model_selection.cross_val_predict(lin,X,Y,cv=cv)
print('MSE:',mean_squared_error(Y,predicted))

print('ridge regression')                         #岭回归
ridge = Ridge(alpha=1.0)
scores = model_selection.cross_val_score(ridge, X, Y, cv=cv)
print(("mean R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(ridge, X,Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))

print('lasso regression')                        #套索回归
lasso = Lasso(alpha=0.1)
scores = model_selection.cross_val_score(lasso, X, Y, cv=cv)
print(("mean R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(lasso, X,Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))

print('decision tree regression')               #决策树
tree = DecisionTreeRegressor(random_state=0)
scores = model_selection.cross_val_score(tree, X, Y, cv=cv)
print(("mean R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(tree, X,Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))
 
print('random forest regression')              #随机森林
forest = RandomForestRegressor(n_estimators=50, max_depth=None,min_samples_split=2, 
                               random_state=0)
scores = model_selection.cross_val_score(forest, X, Y, cv=cv)
print(("mean R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(forest, X,Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))

print('linear support vector machine')        #线性内核的支持向量机
svm_lin = svm.SVR(epsilon=0.2,kernel='linear',C=1)
scores = model_selection.cross_val_score(svm_lin, X, Y, cv=cv)
print(("mean R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(svm_lin, X,Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))

print('support vector machine rbf')            #支持向量机
clf = svm.SVR(epsilon=0.2,kernel='rbf',C=1.)
scores = model_selection.cross_val_score(clf, X, Y, cv=cv)
print(("mean R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(clf, X,Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))

print('knn')                                   #K近邻
knn = KNeighborsRegressor()
scores = model_selection.cross_val_score(knn, X, Y, cv=cv)
print(("mean R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(knn, X,Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))

#R^2（判定系数）的值越接近1，模型的预测效果越好
#随机森林算法训练的模型对数据的拟合程度最高，线性内核支持向量机比普通支持向量机提升效果明显

linear regression
mean R2:0.70 (+/- 0.24)
MSE: 24.157283417962216
ridge regression
mean R2: 0.69 (+/- 0.25)
MSE: 24.35252724752238
lasso regression
mean R2: 0.68 (+/- 0.24)
MSE: 25.386084654728126
decision tree regression
mean R2: 0.79 (+/- 0.14)
MSE: 17.02118577075099
random forest regression
mean R2: 0.87 (+/- 0.14)
MSE: 9.988459351778657
linear support vector machine
mean R2: 0.67 (+/- 0.31)
MSE: 26.350858859089254
support vector machine rbf
mean R2: 0.19 (+/- 0.32)
MSE: 67.50078365079564
knn
mean R2: 0.55 (+/- 0.21)
MSE: 36.57860158102767


In [3]:
#选取特征，递归特征消除法（只选取具有最大绝对权重的特征）
#sklearn库中内置了函数RFE

from sklearn.feature_selection import RFE
best_features = 4                                #设置最佳特征的数量为4

print('feature selection on linear regression')  #线性回归
rfe_lin = RFE(lin,best_features).fit(X,Y)        #调用RFE函数计算
mask = np.array(rfe_lin.support_)                #计算在RFE下模型的判定系数以及均方误差
scores = model_selection.cross_val_score(lin, X[:,mask], Y, cv=cv)
print(("R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(lin, X[:,mask],Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))

print('feature selection ridge regression')      #岭回归
rfe_ridge = RFE(ridge,best_features).fit(X,Y)
mask = np.array(rfe_ridge.support_)
scores = model_selection.cross_val_score(ridge, X[:,mask], Y, cv=cv)
print(("R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(ridge, X[:,mask],Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))

print('feature selection on lasso regression')    #套索回归
rfe_lasso = RFE(lasso,best_features).fit(X,Y)
mask = np.array(rfe_lasso.support_)
scores = model_selection.cross_val_score(lasso, X[:,mask], Y, cv=cv)
print(("R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(lasso, X[:,mask],Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))
 
print('feature selection on decision tree')       #决策树
rfe_tree = RFE(tree,best_features).fit(X,Y)
mask = np.array(rfe_tree.support_)
scores = model_selection.cross_val_score(tree, X[:,mask], Y, cv=cv)
print(("R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(tree, X[:,mask],Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))

print('feature selection on random forest')       #随机森林
rfe_forest = RFE(forest,best_features).fit(X,Y)
mask = np.array(rfe_forest.support_)
scores = model_selection.cross_val_score(forest, X[:,mask], Y, cv=cv)
print(("R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(forest, X[:,mask],Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))

print('feature selection on linear support vector machine')#支持向量机
rfe_svm = RFE(svm_lin,best_features).fit(X,Y)
mask = np.array(rfe_svm.support_)
scores = model_selection.cross_val_score(svm_lin, X[:,mask], Y, cv=cv)
print(("R2: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)))
predicted = model_selection.cross_val_predict(svm_lin, X,Y, cv=cv)
print('MSE:',mean_squared_error(Y,predicted))


feature selection on linear regression


E:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=4 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


R2: 0.57 (+/- 0.44)
MSE: 34.02605152525706
feature selection ridge regression
R2: 0.57 (+/- 0.44)
MSE: 34.09757261001352
feature selection on lasso regression
R2: 0.65 (+/- 0.23)
MSE: 28.07040398875744
feature selection on decision tree


E:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=4 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
E:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=4 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
E:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=4 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


R2: 0.78 (+/- 0.19)
MSE: 17.78276679841897
feature selection on random forest


E:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=4 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


R2: 0.84 (+/- 0.19)
MSE: 12.80952382608696
feature selection on linear support vector machine


E:\anaconda\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_features_to_select=4 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


R2: 0.56 (+/- 0.46)
MSE: 26.350858859089254


In [18]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

df = pd.read_csv('./data/data_cars.csv',header = None)   #导入数据（描述汽车特点的数据集）
for i in range(len(df.columns)):                         #将数据转化为float类型
    df[i] = df[i].astype('category')
print(df.head())
#数据中的数据是相应的描述，所以为了处理要把特征值转换为数字
map0 = dict(zip(df[1].cat.categories,range(len(df[0].cat.categories))))
#zip() 函数用于将可迭代的对象作为参数，将对象中对应的元素打包成一个个元组，然后返回由这些元组组成的列表。
#cat()函数用于合并series
#category是pandas中定义的一个数据类型，相当于R中的因子。可以对特点的类型数据进行按照自己的意愿进行排序
print(map0)
map1 = dict( list(zip( df[1].cat.categories, list(range( len(df[1].cat.categories ))))))
map2 = dict( list(zip( df[2].cat.categories, list(range( len(df[2].cat.categories ))))))
map3 = dict( list(zip( df[3].cat.categories, list(range( len(df[3].cat.categories ))))))
map4 = dict( list(zip( df[4].cat.categories, list(range( len(df[4].cat.categories ))))))
map5 = dict( list(zip( df[5].cat.categories, list(range( len(df[5].cat.categories ))))))
map6 = dict( list(zip( df[6].cat.categories, list(range( len(df[6].cat.categories ))))))

cat_cols = df.select_dtypes(['category']).columns          #选择所有category类型的数据
df[cat_cols] = df[cat_cols].apply(lambda x:x.cat.codes)

df = df.iloc[np.random.permutation(len(df))]               #打乱原数据顺序
print(df.head())
#f值、准确率、召回率用于评估算法的正确性
df_f1 = pd.DataFrame(columns=['method']+sorted(map6, key=map6.get))
df_precision = pd.DataFrame(columns=['method']+sorted(map6, key=map6.get))
df_recall = pd.DataFrame(columns=['method']+sorted(map6, key=map6.get))
#定义函数，将类别标签向量Y和特征向量分开（因为需要计算和保存所有方法分类效果的度量指标）
def CalcMeasures(method,y_pred,y_true,df_f1=df_f1
                 ,df_precision=df_precision,df_recall=df_recall):

    df_f1.loc[len(df_f1)] = [method]+list(f1_score(y_pred,y_true,average=None))
    df_precision.loc[len(df_precision)] = [method]+list(precision_score(y_pred,y_true,average=None))
    df_recall.loc[len(df_recall)] = [method]+list(recall_score(y_pred,y_true,average=None))
 
X = df[df.columns[:-1]].values
Y = df[df.columns[-1]].values
#接下来计算各种算法的f值、准确率、召回率
cv = 10                                           #线性内核支持向量机
method = 'linear support vector machine'
clf = svm.SVC(kernel = 'linear',C=50)
y_pred = model_selection.cross_val_predict(clf,X,Y,cv=cv)
CalcMeasures(method,y_pred,Y)

method = 'rbf support vector machine'            #径向基内核支持向量机         
clf = svm.SVC(kernel='rbf',C=50)
y_pred = model_selection.cross_val_predict(clf, X,Y, cv=cv)
CalcMeasures(method,y_pred,Y)

method = 'poly support vector machine'            #rbf内核支持向量机
clf = svm.SVC(kernel='poly',C=50)
y_pred = model_selection.cross_val_predict(clf, X,Y, cv=cv)
CalcMeasures(method,y_pred,Y)

method = 'decision tree'                          #决策树
clf = DecisionTreeClassifier(random_state=0)
y_pred = model_selection.cross_val_predict(clf, X,Y, cv=cv)
CalcMeasures(method,y_pred,Y)

method = 'random forest'                           #随机森林
clf = RandomForestClassifier(n_estimators=50,random_state=0,max_features=None)
y_pred = model_selection.cross_val_predict(clf, X,Y, cv=cv)
CalcMeasures(method,y_pred,Y)

method = 'naive bayes'                            #朴素贝叶斯
clf = MultinomialNB()
y_pred = model_selection.cross_val_predict(clf, X,Y, cv=cv)
CalcMeasures(method,y_pred,Y)

method = 'logistic regression'                     #逻辑回归
clf = LogisticRegression()
y_pred = model_selection.cross_val_predict(clf, X,Y, cv=cv)
CalcMeasures(method,y_pred,Y)

method = 'k nearest neighbours'                    #K临近
clf = KNeighborsClassifier(weights='distance',n_neighbors=5)
y_pred = model_selection.cross_val_predict(clf, X,Y, cv=cv)
CalcMeasures(method,y_pred,Y)

print(df_f1)
print(df_precision)
print(df_recall)

labels_counts= df[6].value_counts()
pd.Series(map6).map(labels_counts)

       0      1  2  3      4     5      6
0  vhigh  vhigh  2  2  small   low  unacc
1  vhigh  vhigh  2  2  small   med  unacc
2  vhigh  vhigh  2  2  small  high  unacc
3  vhigh  vhigh  2  2    med   low  unacc
4  vhigh  vhigh  2  2    med   med  unacc
{'high': 0, 'low': 1, 'med': 2, 'vhigh': 3}
      0  1  2  3  4  5  6
161   3  0  1  2  0  0  2
1077  2  0  3  2  0  1  2
1039  2  0  2  1  1  2  0
23    3  3  0  2  1  0  2
93    3  3  3  1  1  1  2


E:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                          method       acc      good     unacc     vgood
0  linear support vector machine  0.263566  0.000000  0.845331  0.000000
1     rbf support vector machine  0.994805  0.992701  0.999174  0.992248
2    poly support vector machine  0.791501  0.842105  0.937526  0.809917
3                  decision tree  0.966234  0.942857  0.991304  0.977099
4                  random forest  0.962677  0.942029  0.991701  0.977099
5                    naive bayes  0.040506  0.000000  0.825983  0.000000
6            logistic regression  0.239583  0.000000  0.822616  0.266667
7           k nearest neighbours  0.779292  0.500000  0.953100  0.686275
                          method       acc      good     unacc     vgood
0  linear support vector machine  0.177083  0.000000  0.980165  0.000000
1     rbf support vector machine  0.997396  0.985507  0.999174  0.984615
2    poly support vector machine  0.776042  0.811594  0.948760  0.753846
3                  decision tree  0.968750  0.95652

E:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


acc       384
good       69
unacc    1210
vgood      65
dtype: int64

In [19]:
################################隐马尔可夫########################################
import numpy as np
from copy import copy

class HMM:
    def __init__(self,pi,A,B):    #定义初始化pi（初始状态的概率分布）,A(状态转移矩阵)B（观察概率矩阵）的的值
        self.pi = pi
        self.A = A
        self.B = B
        
    def MostLikelyStateSequence(self,observations):
        
        N = self.A.shape[0]
        T = len(observations)
        sequences = [str(i) for i in range(N)]
        probs = np.array([self.pi[i]*self.B[i,observations[0]] for i in range(N)])
        print(probs)
        for i in range(1,T):
            newsequences = []
            newprobs = np.array([])
            for s in range(len(sequences)):
                for j in range(N):
                    newsequences.append(sequences[s]+str(j))
                    bef = int(sequences[s][-1])
                    tTpprob = probs[s]*self.A[bef,j]*self.B[j,observations[i]]
                    newprobs = np.append(newprobs,[tTpprob]) 
                    print(sequences[s]+str(j),'-',tTpprob)
            sequences = newsequences
            probs = newprobs
        return max((probs[i],sequences[i]) for i in range(len(sequences)))
            
    def ViterbiSequence(self,observations):
        deltas = [{}]
        seq = {}
        N = self.A.shape[0]
        states = [i for i in range(N)]
        T = len(observations)
        #initialization
        for s in states:
            deltas[0][s] = self.pi[s]*self.B[s,observations[0]]
            seq[s] = [s]
        #compute Viterbi
        for t in range(1,T):
            deltas.append({})
            newseq = {}
            for s in states:
                (delta,state) = max((deltas[t-1][s0]*self.A[s0,s]*self.B[s,observations[t]],s0) for s0 in states)
                deltas[t][s] = delta
                newseq[s] = seq[state] + [s]
            seq = newseq
            
        (delta,state) = max((deltas[T-1][s],s) for s in states)
        return  delta,' sequence: ', seq[state]
        
    def maxProbSequence(self,observations):
        N = self.A.shape[0]
        states = [i for i in range(N)]
        T = len(observations)
        M = self.B.shape[1]
        # alpha_t(i) = P(O_1 O_2 ... O_t, q_t = S_i | hmm)
        # Initialize alpha
        alpha = np.zeros((N,T))
        c = np.zeros(T) #scale factors
        alpha[:,0] = pi.T * self.B[:,observations[0]]
        c[0] = 1.0/np.sum(alpha[:,0])
        alpha[:,0] = c[0] * alpha[:,0]
        # Update alpha for each observation step
        for t in range(1,T):
            alpha[:,t] = np.dot(alpha[:,t-1].T, self.A).T * self.B[:,observations[t]]
            c[t] = 1.0/np.sum(alpha[:,t])
            alpha[:,t] = c[t] * alpha[:,t]

        # beta_t(i) = P(O_t+1 O_t+2 ... O_T | q_t = S_i , hmm)
        # Initialize beta
        beta = np.zeros((N,T))
        beta[:,T-1] = 1
        beta[:,T-1] = c[T-1] * beta[:,T-1]
        # Update beta backwards froT end of sequence
        for t in range(len(observations)-1,0,-1):
            beta[:,t-1] = np.dot(self.A, (self.B[:,observations[t]] * beta[:,t]))
            beta[:,t-1] = c[t-1] * beta[:,t-1]
            
        norm = np.sum(alpha[:,T-1])
        seq = ''
        for t in range(T):
            g,state = max(((beta[i,t]*alpha[i,t])/norm,i) for i in states)
            seq +=str(state)
            
        return seq
        
    def simulate(self,time):

        def drawFromNormal(probs):
            return np.where(np.random.multinomial(1,probs) == 1)[0][0]

        observations = np.zeros(time)
        states = np.zeros(time)
        states[0] = drawFromNormal(self.pi)
        observations[0] = drawFromNormal(self.B[states[0],:])
        for t in range(1,time):
            states[t] = drawFromNormal(self.A[states[t-1],:])
            observations[t] = drawFromNormal(self.B[states[t],:])
        return observations,states


    def train(self,observations,criterion):

        N = self.A.shape[0]
        T = len(observations)
        M = self.B.shape[1]

        A = self.A
        B = self.B
        pi = copy(self.pi)
        
        convergence = False
        while not convergence:

            # alpha_t(i) = P(O_1 O_2 ... O_t, q_t = S_i | hmm)
            # Initialize alpha
            alpha = np.zeros((N,T))
            c = np.zeros(T) #scale factors
            alpha[:,0] = pi.T * self.B[:,observations[0]]
            c[0] = 1.0/np.sum(alpha[:,0])
            alpha[:,0] = c[0] * alpha[:,0]
            # Update alpha for each observation step
            for t in range(1,T):
                alpha[:,t] = np.dot(alpha[:,t-1].T, self.A).T * self.B[:,observations[t]]
                c[t] = 1.0/np.sum(alpha[:,t])
                alpha[:,t] = c[t] * alpha[:,t]

            #P(O=O_0,O_1,...,O_T-1 | hmm)
            P_O = np.sum(alpha[:,T-1])
            # beta_t(i) = P(O_t+1 O_t+2 ... O_T | q_t = S_i , hmm)
            # Initialize beta
            beta = np.zeros((N,T))
            beta[:,T-1] = 1
            beta[:,T-1] = c[T-1] * beta[:,T-1]
            # Update beta backwards froT end of sequence
            for t in range(len(observations)-1,0,-1):
                beta[:,t-1] = np.dot(self.A, (self.B[:,observations[t]] * beta[:,t]))
                beta[:,t-1] = c[t-1] * beta[:,t-1]

            gi = np.zeros((N,N,T-1));

            for t in range(T-1):
                for i in range(N):
                    
                    gamma_num = alpha[i,t] * self.A[i,:] * self.B[:,observations[t+1]].T * \
                            beta[:,t+1].T
                    gi[i,:,t] = gamma_num / P_O
  
            # gamma_t(i) = P(q_t = S_i | O, hmm)
            gamma = np.squeeze(np.sum(gi,axis=1))
            # Need final gamma element for new B
            prod =  (alpha[:,T-1] * beta[:,T-1]).reshape((-1,1))
            gamma_T = prod/P_O
            gamma = np.hstack((gamma,  gamma_T)) #append one Tore to gamma!!!

            newpi = gamma[:,0]
            newA = np.sum(gi,2) / np.sum(gamma[:,:-1],axis=1).reshape((-1,1))
            newB = copy(B)
            
            sumgamma = np.sum(gamma,axis=1)
            for ob_k in range(M):
                list_k = observations == ob_k
                newB[:,ob_k] = np.sum(gamma[:,list_k],axis=1) / sumgamma

            if np.max(abs(pi - newpi)) < criterion and \
                   np.max(abs(A - newA)) < criterion and \
                   np.max(abs(B - newB)) < criterion:
                convergence = True;
  
            A[:],B[:],pi[:] = newA,newB,newpi

        self.A[:] = newA
        self.B[:] = newB
        self.pi[:] = newpi
        self.gamma = gamma
        

if __name__ == '__main__':
       
    pi = np.array([0.6, 0.4])
    A = np.array([[0.7, 0.3],[0.6, 0.4]])
    B = np.array([[0.7, 0.1, 0.2],[0.1, 0.6, 0.3]])
    hmmguess = HMM(pi,A,B)
    print('Viterbi sequence:',hmmguess.ViterbiSequence(np.array([0,1,0,2])))
    print('max prob sequence:',hmmguess.maxProbSequence(np.array([0,1,0,2])))    
    #obs,states = hmmguess.simulate(4)
    hmmguess.train(np.array([0,1,0,2]),0.000001)

    print('Estimated initial probabilities:',hmmguess.pi)
    print('Estimated state transition probabililities:',hmmguess.A)
    print('Estimated observation probabililities:',hmmguess.B)

Viterbi sequence: (0.004445279999999999, ' sequence: ', [0, 1, 0, 0])
max prob sequence: 0100
Estimated initial probabilities: [1. 0.]
Estimated state transition probabililities: [[0. 1.]
 [1. 0.]]
Estimated observation probabililities: [[1.         0.         0.        ]
 [0.         0.38196618 0.61803382]]
